In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import networkx as nx
import music21 as msc
from funciones import *

env = msc.environment.UserSettings()
#env['musicxmlPath'] = r'C:\Program Files (x86)\Finale NotePad 2012\Finale NotePad.exe' #Path a la aplicacion Finale
#env['musicxmlPath'] = r'C:\Program Files (x86)\MuseScore 2\bin\MuseScore.exe'          #Path a la aplicacion MuseScore 
#env['musicxmlPath']='/usr/bin/mscore'
msc.environment.set('musescoreDirectPNGPath', 'C:\\Program Files (x86)\\MuseScore 2\\bin\\MuseScore.exe')
msc.environment.set('musicxmlPath', 'C:\\Program Files (x86)\\MuseScore 2\\bin\\MuseScore.exe')


In [3]:
#Importo un xml de prueba

#G=f_xml2graph(r'.\\partituras\\Queen_-_Bohemian_Rhapsody.mxl')
#G=f_xml2graph(r'.\\partituras\\Here_Comes_The_Sun_-_The_Beatles.mscz.mxl')
#G=f_xml2graph(r'.\\partituras\\Mozart__Rondo_Ala_Turca.mxl')
#G=f_xml2graph(r'.\\partituras\\Chopin_-_Nocturne_Op_9_No_2_E_Flat_Major.mxl')
#G=f_xml2graph(r'.\\partituras\\Bach_Toccata_and_Fugue_in_D_Minor_Piano_solo.mxl')
#G=f_xml2graph(r'.\\partituras\\bachcelloprelude.mxl')
G=f_xml2graph(r'.\\partituras\\queen_underpressure.mxl', 'Bass Guitar')


#song = msc.converter.parse(r'.\\partituras\\13725_ice_ice_baby.mxl') # Lee la cancion, queda un elemento stream.Score

#Lp = len(song.parts) # Cantidad de partes (voces)
#lista_partes = list(np.zeros(Lp)) # Crea una lista donde se van a guardas los nombres de las partes

#for i,elem in enumerate(song.parts):
#        lista_partes[i] = elem.partName

                
#G=f_xml2graph(r'.\\partituras\\13725_ice_ice_baby.mxl', lista_partes[1])


#--------------------------------------------------
#Canción de prueba

#sBach = msc.corpus.parse('bach/bwv57.8')
#sBach.show()

Parte seleccionada: Bass Guitar


In [ ]:
f_grado_dist_M(G)
f_grado_dist_H(G)
f_tabla(G,'artista')